In [1]:
from xbbbNN import visualize, get_xb_bb_positions, get_network_features, small_network_evaluation, head_on_network_evaluation

2024-01-03 13:52:42.263525: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-01-03 13:52:50.106518: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-01-03 13:53:08.607285: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: :/home/highgarden/opt/openmpi/lib
2024-01-03 13:53:08.607416: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so

In [2]:
model_path = "/home/highgarden/Workspace/Thesis/XB_BB_Docking_Score/Data/Neural_Network/Weights/2,2,2-4,1-f2-it1"

In [3]:
import pymolviz as pmv
import numpy as np

In [4]:
import tensorflow as tf
import xbbbNN
tf.config.set_visible_devices([], 'GPU')
visible_devices = tf.config.get_visible_devices()
for device in visible_devices:
    assert device.device_type != 'GPU'

2024-01-03 13:53:31.978482: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:966] could not open file to read NUMA node: /sys/bus/pci/devices/0000:2b:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-01-03 13:53:32.441783: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:966] could not open file to read NUMA node: /sys/bus/pci/devices/0000:2b:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-01-03 13:53:32.442054: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:966] could not open file to read NUMA node: /sys/bus/pci/devices/0000:2b:00.0/numa_node
Your kernel may have been built without NUMA support.


In [5]:
model = tf.keras.models.load_model(model_path, compile=False)

2024-01-03 13:53:32.917376: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [6]:
from xbpy import rdutil

In [7]:
def direct_network_application(halogen_position, orientation, acceptor_atoms, halogen_symbol, bond_distance):
    head_on_predictions = [0.0]
    side_on_predictions = [0.0]
    for acceptor_atom in acceptor_atoms:
            try:
                acceptor_positions = get_xb_bb_positions(acceptor_atom)
            except IndexError:
                continue
            carbon_position = halogen_position - (orientation * bond_distance)
            head_on_input, side_on_input = get_network_features(halogen_symbol, halogen_position, carbon_position, acceptor_positions)
            prediction = model.predict([np.zeros((1, 114)), np.zeros((1, 1)), side_on_input.reshape(1, 19), head_on_input.reshape(1, 9)], verbose=0)
            side_on_predictions.append(prediction[0][1])
            head_on_predictions.append(prediction[0][2])
    return np.max(head_on_predictions)
            

In [8]:
pocket = rdutil.get_binding_pockets_by_ligand(rdutil.read_molecules("5nkd.pdb", proximityBonding=True)[0])[0]

In [9]:
all_detected = []
for oxygen in pocket.pocket_atoms:
    if not oxygen.GetSymbol() == "O":
        continue
    try:
        positions = get_xb_bb_positions(oxygen)
        #print(oxygen.GetIdx())
        all_detected.append(pmv.Points(positions, color = ["O", "C", "C", "H", "N", "H", "C", "H"], name = f"O{oxygen.GetIdx()}"))
    except:
        continue

pmv.Group(all_detected, name = "detected_acceptors").write("detected_acceptors.py")

In [10]:

visualize(direct_network_application, "5nkd.pdb").write("direct_head_on_test.py")
visualize(head_on_network_evaluation, "5nkd.pdb").write("head_on_test.py")

100%|██████████| 13/13 [02:18<00:00, 10.65s/it]


[0.0, 0.0, 0.0, 1.7678719758987427, 0.0, 0.6337112784385681, 0.3231920301914215, 0.0, 0.01786426454782486, 0.43778762221336365, 1.3593761920928955, 1.06415855884552, 0.0]


IndexError: index 0 is out of bounds for axis 0 with size 0

In [ ]:
visualize(small_network_evaluation, "5nkd.pdb").write("test.py")

 20%|█▉        | 338/1720 [04:27<18:55,  1.22it/s]